In [8]:
%matplotlib inline

from consts import *
import os,cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon, Rectangle
from PIL import Image
from keras.models import Model
from joblib import dump, load

dest_dir = base_path + 'images/rcnn_test/'



In [10]:
# load up the CNN weights
weights_dir = base_path + 'weights/'

from CNN_model import model

model.load_weights(weights_dir + 'cnn_' + str(CLASS_ID) + '.h5')
intermed_model = Model(inputs=model.input,
                   outputs=model.layers[-2].output)
intermed_model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
dropout (Dropout)            (None, 21, 21, 64)        0   

In [11]:
# loading up the SVM
clf = load(svm_weights_path) 

In [ ]:
img_path = base_path + 'images/test14.jpg'
# run selective search
image = cv2.imread(img_path)[...,::-1] # switch to rgb

# initialise selective search
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

ss.setBaseImage(image)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()
print(len(ssresults))

In [ ]:
if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)

imout = image.copy()
plt.figure(figsize=(18,18))
ax = plt.gca()
ax.imshow(image)

n = 0
for i,result in enumerate(ssresults):
    if i < 2000: 
        x,y,w,h = result
        if w < 50 or h < 50:
            continue
        crop = imout[y:y+h,x:x+w]
        resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)
        # resized = np.array(resized)
        resized_norm = (resized/256) - 0.5
        im = np.expand_dims(resized_norm, axis=0)
        layer_output = intermed_model(im)

        svm_prob = clf.predict_proba(layer_output)
        # print(svm_prob)
        if svm_prob[0][1] > 0.999:
            pil_img = Image.fromarray(resized)
            pil_img.save(dest_dir + 'negative_' + str(n) +'.png')
            n += 1
            rect = Rectangle([x ,y], w, h, fill=False, 
                                color='red', lw=1) 
            ax.add_patch(rect)
plt.show()